In [1]:
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

/Users/bratu/Desktop/dsp/venv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
with open('auto-mpg.csv', 'r') as fp:
    df = pd.read_csv(fp, na_values='?')

In [3]:
df_clean = df.drop(columns=['car name'])
df_clean.dropna()
df_clean['origin'] = df_clean['origin'].apply(lambda x: {1: 'USA', 2: 'Europe', 3: 'Japan'}[x])
df_clean

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,18.0,8,307.0,130.0,3504,12.0,70,USA
1,15.0,8,350.0,165.0,3693,11.5,70,USA
2,18.0,8,318.0,150.0,3436,11.0,70,USA
3,16.0,8,304.0,150.0,3433,12.0,70,USA
4,17.0,8,302.0,140.0,3449,10.5,70,USA
...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,USA
394,44.0,4,97.0,52.0,2130,24.6,82,Europe
395,32.0,4,135.0,84.0,2295,11.6,82,USA
396,28.0,4,120.0,79.0,2625,18.6,82,USA


In [4]:
origin_ohe = pd.get_dummies(df_clean['origin'])
origin_ohe

,Europe,Japan,USA
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
393,0,0,1
394,1,0,0
395,0,0,1
396,0,0,1


In [5]:
df_ohe = pd.concat([df_clean, origin_ohe], axis=1).drop(columns=['origin'])
df_ohe

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,Europe,Japan,USA
0,18.0,8,307.0,130.0,3504,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,0,0,1
...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,0,0,1
394,44.0,4,97.0,52.0,2130,24.6,82,1,0,0
395,32.0,4,135.0,84.0,2295,11.6,82,0,0,1
396,28.0,4,120.0,79.0,2625,18.6,82,0,0,1


In [6]:
df_final = df_ohe.copy()
for col in ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'Europe', 'Japan', 'USA']:
    scaler = StandardScaler()
    df_final[col] = scaler.fit_transform(df_final[col].values.reshape(-1, 1))
df_final

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,Europe,Japan,USA
0,18.0,1.498191,1.090604,0.664133,0.630870,-1.295498,-1.627426,-0.461968,-0.497643,0.773559
1,15.0,1.498191,1.503514,1.574594,0.854333,-1.477038,-1.627426,-0.461968,-0.497643,0.773559
2,18.0,1.498191,1.196232,1.184397,0.550470,-1.658577,-1.627426,-0.461968,-0.497643,0.773559
3,16.0,1.498191,1.061796,1.184397,0.546923,-1.295498,-1.627426,-0.461968,-0.497643,0.773559
4,17.0,1.498191,1.042591,0.924265,0.565841,-1.840117,-1.627426,-0.461968,-0.497643,0.773559
...,...,...,...,...,...,...,...,...,...,...
393,27.0,-0.856321,-0.513026,-0.480448,-0.213324,0.011586,1.621983,-0.461968,-0.497643,0.773559
394,44.0,-0.856321,-0.925936,-1.364896,-0.993671,3.279296,1.621983,2.164651,-0.497643,-1.292726
395,32.0,-0.856321,-0.561039,-0.532474,-0.798585,-1.440730,1.621983,-0.461968,-0.497643,0.773559
396,28.0,-0.856321,-0.705077,-0.662540,-0.408411,1.100822,1.621983,-0.461968,-0.497643,0.773559


In [7]:
X = df_final.drop(columns=['mpg']).to_numpy()
y = df_final['mpg'].to_numpy()

In [8]:
model = XGBRegressor(seed=42, random_state=42, colsample_bytree=1)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = np.absolute(scores)
print('Mean MAE %.3f STD MAE %.3f' % (scores.mean(), scores.std()) )

/Users/bratu/Desktop/dsp/venv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/bratu/Desktop/dsp/venv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/bratu/Desktop/dsp/venv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/bratu/Desktop/dsp/venv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a futur

Mean MAE 1.995 STD MAE 0.338


In [9]:
with open('best.csv', 'w+') as fp:
    df_final.to_csv(fp)

In [10]:
df_final

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,Europe,Japan,USA
0,18.0,1.498191,1.090604,0.664133,0.630870,-1.295498,-1.627426,-0.461968,-0.497643,0.773559
1,15.0,1.498191,1.503514,1.574594,0.854333,-1.477038,-1.627426,-0.461968,-0.497643,0.773559
2,18.0,1.498191,1.196232,1.184397,0.550470,-1.658577,-1.627426,-0.461968,-0.497643,0.773559
3,16.0,1.498191,1.061796,1.184397,0.546923,-1.295498,-1.627426,-0.461968,-0.497643,0.773559
4,17.0,1.498191,1.042591,0.924265,0.565841,-1.840117,-1.627426,-0.461968,-0.497643,0.773559
...,...,...,...,...,...,...,...,...,...,...
393,27.0,-0.856321,-0.513026,-0.480448,-0.213324,0.011586,1.621983,-0.461968,-0.497643,0.773559
394,44.0,-0.856321,-0.925936,-1.364896,-0.993671,3.279296,1.621983,2.164651,-0.497643,-1.292726
395,32.0,-0.856321,-0.561039,-0.532474,-0.798585,-1.440730,1.621983,-0.461968,-0.497643,0.773559
396,28.0,-0.856321,-0.705077,-0.662540,-0.408411,1.100822,1.621983,-0.461968,-0.497643,0.773559
